# Document Summarization


This notebook demonstrates an application of long document summarization techniques to a work of literature using Granite.

## Install Dependencies


Granite utils provides some helpful functions for recipes.

In [2]:
! echo "::group::Install Dependencies"
%pip install uv
! uv pip install git+https://github.com/ibm-granite-community/utils.git \
    transformers \
    langchain_ollama \
    'langchain_replicate @ git+https://github.com/ibm-granite-community/langchain-replicate.git' \
    docling
! echo "::endgroup::"

::group::Install Dependencies
Using Python 3.12.12 environment at: /usr
Resolved 121 packages in 411ms
Audited 121 packages in 2ms
::endgroup::


## Serving the Granite AI model


This notebook requires IBM Granite models to be served by an AI model runtime so that the models can be invoked or called. This notebook can use a locally accessible [Ollama](https://github.com/ollama/ollama) server to serve the models, or the [Replicate](https://replicate.com) cloud service.

During the pre-work, you may have either started a local Ollama server on your computer, or setup Replicate access and obtained an [API token](https://replicate.com/account/api-tokens).

## Select your model

Select a Granite model to use. Here we use a Langchain client to connect to the model. If there is a locally accessible Ollama server, we use an Ollama client to access the model. Otherwise, we use a Replicate client to access the model.

When using Replicate, if the `REPLICATE_API_TOKEN` environment variable is not set, or a `REPLICATE_API_TOKEN` Colab secret is not set, then the notebook will ask for your [Replicate API token](https://replicate.com/account/api-tokens) in a dialog box.

In [3]:
import os
import requests
from langchain_ollama import ChatOllama
from langchain_replicate import ChatReplicate
from ibm_granite_community.notebook_utils import get_env_var

model_path = "ibm-granite/granite-4.0-h-small"
try: # Look for a locally accessible Ollama server for the model
    response = requests.get(os.getenv("OLLAMA_HOST", "http://127.0.0.1:11434"))
    model = ChatOllama(
        model="ibm/granite4:micro",
        num_ctx=65536, # 64K context window
        num_predict=2000, # Set the maximum number of tokens to generate as output.
        temperature=0.0,
    )
except Exception: # Use Replicate for the model
    model = ChatReplicate(
        model=model_path,
        replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
        model_kwargs={
            "max_tokens": 2000, # Set the maximum number of tokens to generate as output.
            "min_tokens": 200, # Set the minimum number of tokens to generate as output.
            "temperature": 0.0,
            "presence_penalty": 0,
            "frequency_penalty": 0,
        },
    )

REPLICATE_API_TOKEN loaded from Google Colab secret.


## Split a book into chunks

The code in the following sections will fetch H.D. Thoreau's "Walden" from [Project Gutenberg](https://www.gutenberg.org/) for summarization.

We will then chunk the book text so that chunks fit in the context window size of the AI model.

### Count the tokens

Before sending our book chunks to the AI model, it's crucial to understand how much of the model's capacity we're using. Language models typically have a limit on the number of tokens they can process in a single request.

Key points:
- We're using the [`granite-3.3`](https://huggingface.co/ibm-granite/granite-3.3-2b-instruct) model, which has a context window of 128K tokens.
- Tokenization can vary between models, so we use the specific tokenizer for our chosen model.

Understanding token count helps us optimize our prompts and ensure we're using the model efficiently.

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

### Summary of Summaries

Here we use a hierarchical abstractive summarization technique to adapt to the context length of the model. Our approach uses [Docling](https://docling-project.github.io/docling/) to understand the document's structure, chunk the document into text passages, and group the text passages by chapter which we can then summarize.

In [5]:
import itertools
from typing import Iterator, Callable
from docling.document_converter import DocumentConverter
from docling_core.transforms.chunker.hierarchical_chunker import HierarchicalChunker
from docling_core.transforms.chunker.base import BaseChunk

def chunk_document(source: str, *, dropwhile: Callable[[BaseChunk], bool] = lambda c: False, takewhile: Callable[[BaseChunk], bool] = lambda c: True) -> Iterator[BaseChunk]:
    """Read the document and perform a hierarchical chunking"""
    converter = DocumentConverter()
    chunks = HierarchicalChunker().chunk(converter.convert(source=source).document)
    return itertools.takewhile(takewhile, itertools.dropwhile(dropwhile, chunks))

def merge_chunks(chunks: Iterator[BaseChunk], *, headings: Callable[[BaseChunk], list[str]] = lambda c: c.meta.headings) -> Iterator[dict[str, str]]:
    """Merge chunks having the same headings"""
    prior_headings: list[str] | None = None
    document: dict[str, str] = {}
    doc_id = 0
    for chunk in chunks:
        text = chunk.text.replace('\r\n', '\n')
        current_headings = headings(chunk)
        if prior_headings != current_headings:
            if document:
                yield document
            prior_headings = current_headings
            document = {
                'doc_id': str(doc_id := doc_id + 1),
                'title': " - ".join(current_headings),
                'text': text
            }
        else:
            document['text'] += f"\n\n{text}"
    if document:
        yield document

def chunk_dropwhile(chunk: BaseChunk) -> bool:
    """Ignore front matter prior to the book start"""
    return "WALDEN" not in chunk.meta.headings

def chunk_takewhile(chunk: BaseChunk) -> bool:
    """Ignore remaining chunks once we see this heading"""
    return "ON THE DUTY OF CIVIL DISOBEDIENCE" not in chunk.meta.headings

def chunk_headings(chunk: BaseChunk) -> list[str]:
    """Use the h1 and h2 (chapter) headings"""
    return chunk.meta.headings[:2]

documents: list[dict[str, str]] = list(merge_chunks(
    chunk_document(
        "https://www.gutenberg.org/cache/epub/205/pg205-images.html",
        dropwhile=chunk_dropwhile,
        takewhile=chunk_takewhile,
    ),
    headings=chunk_headings,
))

print(f"{len(documents)} documents created")
print(f"Max document size: {max(len(tokenizer.tokenize(document['text'])) for document in documents)} tokens")

19 documents created
Max document size: 33510 tokens


## Summarize the chunks

Here we define a method to generate a response using a list of documents and a user prompt about those documents.

We create the prompt according to the [Granite Prompting Guide](https://www.ibm.com/granite/docs/models/granite/#basic-chat-template-example) and provide the documents using the `documents` parameter.

In [6]:
from ibm_granite_community.langchain.prompts import TokenizerChatPromptTemplate
from ibm_granite_community.langchain.utils import add_document_role_messages

prompt_template = TokenizerChatPromptTemplate.from_template("{user_prompt}", tokenizer=tokenizer)

def generate(user_prompt: str, documents: list[dict[str, str]]) -> str:
    """Use the chat template to format the prompt"""
    prompt = prompt_template.format_prompt(user_prompt=user_prompt, documents=documents)

    print(f"Input size: {len(tokenizer.tokenize(prompt.to_string()))} tokens")
    if isinstance(model, ChatOllama):  # Ollama does not accept documents parameter
        output = model.invoke(add_document_role_messages(prompt.to_messages(), documents))
    else:
        output = model.invoke(prompt, documents=documents)
    print(f"Output size: {len(tokenizer.tokenize(output.text()))} tokens")

    return output.text()

For each chapter, we create a separate summary. This can take a few minutes.

In [9]:
from collections import defaultdict

In [10]:
summaries: defaultdict(dict[str,str]) = {} #[dict[str, str]] = []

In [12]:
if get_env_var('GRANITE_TESTING', 'false').lower() == 'true':
    documents = documents[:5] # shorten testing work

user_prompt = """\
Using only the the book chapter document, compose a summary of the book chapter.
Your response should only include the summary. Do not provide any further explanation."""

for i, document in enumerate(documents, start=1):
    print(f"============================= {document['title']} ({i}/{len(documents)}) =============================")
    output = generate(user_prompt, [document])
    if i not in summaries.keys():
      summaries[i] = {
          'doc_id': document['doc_id'],
          'title': document['title'],
          'text': output,
      }
      print("Summary: " + output)

print("Summary count: " + str(len(summaries)))

============================= WALDEN - by Henry David Thoreau (1/19) =============================
Input size: 166 tokens


/tmp/ipython-input-3320625964.py:10: LangChainDeprecationWarning: Calling .text() as a method is deprecated. Use .text as a property instead (e.g., message.text).
  output = generate(user_prompt, [document])


Output size: 901 tokens
Summary: The chapter "Walden" by Henry David Thoreau is a philosophical exploration of simple living in natural surroundings. Thoreau describes his two-year experiment of living in a small cabin near Walden Pond, where he sought to simplify his life and focus on the essentials. He reflects on the beauty of nature, the importance of self-reliance, and the value of solitude. Thoreau also critiques the materialism and conformity of society, arguing that people should live deliberately and pursue their own truths. The chapter is a call to simplify one's life, to focus on what is truly important, and to live in harmony with nature. It is a timeless work that continues to inspire readers to this day. The chapter is divided into several sections, each exploring a different aspect of Thoreau's experiment. In the first section, "Economy," Thoreau describes the practical aspects of his experiment, including the cost of building his cabin and the sources of his food. He ar

## Create the Final Summary

Now we need to summarize the chapter summaries. We prompt the model to create a unified summary of the chapter summaries we previously generated.

In [14]:
from ibm_granite_community.notebook_utils import wrap_text

user_prompt = """\
Using only the book chapter summary documents, compose a single, unified summary of the book.
Your response should only include the unified summary. Do not provide any further explanation."""


summaries_list = list(summaries.values())
output = generate(user_prompt, summaries_list)
print(wrap_text(output))

Input size: 5628 tokens
Output size: 857 tokens
"Walden" by Henry David Thoreau is a philosophical exploration of simple living
in natural surroundings. Thoreau describes his two-year experiment of living in
a small cabin near Walden Pond, where he sought to simplify his life and focus
on the essentials. He reflects on the beauty of nature, the importance of self-
reliance, and the value of solitude. Thoreau also critiques the materialism and
conformity of society, arguing that people should live deliberately and pursue
their own truths. The chapter is divided into several sections, each exploring a
different aspect of Thoreau's experiment. In the first section, "Economy,"
Thoreau describes the practical aspects of his experiment, including the cost of
building his cabin and the sources of his food. He argues that people should be
mindful of their expenses and live within their means. In the second section,
"Where I Lived, and What I Lived For," Thoreau reflects on the reasons for his


/tmp/ipython-input-737530950.py:9: LangChainDeprecationWarning: Calling .text() as a method is deprecated. Use .text as a property instead (e.g., message.text).
  output = generate(user_prompt, summaries_list)


So we have now summarized a document larger than the AI model's context window length by breaking the document down into smaller pieces to summarize and then summarizing those summaries.